This week, you will be making and implementing decisions about data management for the variables that you have chosen to examine based on the steps reviewed in the videos and supplemental materials. This assignment is important because it offers you the opportunity to practice making sound data management decisions and think about how these decisions will impact your research. 

STEP 1: Make and implement data management decisions for the variables you selected.

Data management includes such things as coding out missing data, coding in valid data, recoding variables, creating secondary variables and binning or grouping variables. Not everyone does all of these, but some is required.

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import datetime
from datetime import datetime, timedelta
import scipy.stats
import pandas_profiling
from pandas_profiling import ProfileReport


%matplotlib inline
#sets the default autosave frequency in seconds
%autosave 60 
sns.set_style('dark')
sns.set(font_scale=1.2)

plt.rc('axes', titlesize=9)
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings('ignore')

# Use Folium library to plot values on a map.
#import folium

import feature_engine.missing_data_imputers as mdi
#from feature_engine.outlier_removers import Winsorizer
#from feature_engine import categorical_encoders as ce

pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.option_context('float_format','{:.2f}'.format)

np.random.seed(0)
np.set_printoptions(suppress=True)

Autosaving every 60 seconds


In [2]:
df = pd.read_csv("gapmindercleaned.csv")

In [3]:
df

,country,incomeperperson,alcconsumption,armedforcesrate,breastcancerper100th,co2emissions,femaleemployrate,hivrate,internetuserate,lifeexpectancy,oilperperson,polityscore,relectricperperson,suicideper100th,employrate,urbanrate
0,Afghanistan,NaN,0.03,0.57,26.8,7.594400e+07,25.6,NaN,3.65,48.67,NaN,0.0,NaN,6.68,55.7,24.04
1,Albania,1915.00,7.29,1.02,57.4,2.237473e+08,42.1,NaN,44.99,76.92,NaN,9.0,636.34,7.70,51.4,46.72
2,Algeria,2231.99,0.69,2.31,23.5,2.932109e+09,31.7,0.1,12.50,73.13,0.42,2.0,590.51,4.85,50.5,65.22
3,Andorra,21943.34,10.17,NaN,NaN,NaN,NaN,NaN,81.00,NaN,NaN,NaN,NaN,5.36,NaN,88.92
4,Angola,1381.00,5.57,1.46,23.1,2.483580e+08,69.4,2.0,10.00,51.09,NaN,-2.0,173.00,14.55,75.7,56.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Vietnam,722.81,3.91,1.09,16.2,1.425435e+09,67.6,0.4,27.85,75.18,NaN,-7.0,302.73,11.65,71.0,27.84
209,West Bank and Gaza,NaN,NaN,5.94,NaN,1.424133e+07,11.3,NaN,36.42,72.83,NaN,NaN,NaN,NaN,32.0,71.90
210,"Yemen, Rep.",610.36,0.20,2.32,35.1,2.348647e+08,20.3,NaN,12.35,65.49,NaN,-2.0,130.06,6.27,39.0,30.64
211,Zambia,432.23,3.56,0.34,13.0,1.320257e+08,53.5,13.5,10.12,49.03,NaN,7.0,168.62,12.02,61.0,35.42


## Exploratory Data Analysis

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   country               213 non-null    object 
 1   incomeperperson       190 non-null    float64
 2   alcconsumption        187 non-null    float64
 3   armedforcesrate       164 non-null    float64
 4   breastcancerper100th  173 non-null    float64
 5   co2emissions          200 non-null    float64
 6   femaleemployrate      178 non-null    float64
 7   hivrate               147 non-null    float64
 8   internetuserate       192 non-null    float64
 9   lifeexpectancy        191 non-null    float64
 10  oilperperson          63 non-null     float64
 11  polityscore           161 non-null    float64
 12  relectricperperson    136 non-null    float64
 13  suicideper100th       191 non-null    float64
 14  employrate            178 non-null    float64
 15  urbanrate             2

In [5]:
df.describe()

,incomeperperson,alcconsumption,armedforcesrate,breastcancerper100th,co2emissions,femaleemployrate,hivrate,internetuserate,lifeexpectancy,oilperperson,polityscore,relectricperperson,suicideper100th,employrate,urbanrate
count,190.000000,187.000000,164.000000,173.000000,2.000000e+02,178.000000,147.000000,192.000000,191.000000,63.000000,161.000000,136.000000,191.000000,178.000000,203.000000
mean,8740.965895,6.689412,1.443963,37.402890,5.033262e+09,47.549438,1.935442,35.632760,69.753979,1.484603,3.689441,1173.179044,9.640419,58.635955,56.769360
std,14262.809055,4.899617,1.709174,22.697901,2.573812e+10,14.625743,4.376727,27.780433,9.708542,1.825321,6.314899,1681.440171,6.299770,10.519455,23.844933
min,103.780000,0.030000,0.000000,3.900000,1.320000e+05,11.300000,0.060000,0.210000,47.790000,0.030000,-10.000000,0.000000,0.200000,32.000000,10.400000
25%,748.245000,2.625000,0.477500,20.600000,3.484617e+07,38.725000,0.100000,10.000000,64.450000,0.530000,-2.000000,203.647500,4.985000,51.225000,36.830000
50%,2553.495000,5.920000,0.930000,30.000000,1.859018e+08,47.550000,0.400000,31.810000,73.130000,1.030000,6.000000,597.135000,8.260000,58.700000,57.940000
75%,9379.895000,9.925000,1.612500,50.300000,1.846084e+09,55.875000,1.300000,56.415000,76.595000,1.625000,9.000000,1491.147500,12.330000,64.975000,74.210000
max,105147.440000,23.010000,10.640000,101.100000,3.342209e+11,83.300000,25.900000,95.640000,83.390000,12.230000,10.000000,11154.760000,35.750000,83.200000,100.000000


In [6]:
df.columns

Index(['country', 'incomeperperson', 'alcconsumption', 'armedforcesrate', 'breastcancerper100th', 'co2emissions', 'femaleemployrate', 'hivrate', 'internetuserate', 'lifeexpectancy', 'oilperperson', 'polityscore', 'relectricperperson', 'suicideper100th', 'employrate', 'urbanrate'], dtype='object')

## Data Preprocessing

### Treat Missing Values

In [7]:
df.isnull().sum()

country                   0
incomeperperson          23
alcconsumption           26
armedforcesrate          49
breastcancerper100th     40
co2emissions             13
femaleemployrate         35
hivrate                  66
internetuserate          21
lifeexpectancy           22
oilperperson            150
polityscore              52
relectricperperson       77
suicideper100th          22
employrate               35
urbanrate                10
dtype: int64

In [8]:
imputer = mdi.MeanMedianImputer(imputation_method='mean',variables=None)

In [9]:
imputer.fit(df)

MeanMedianImputer(imputation_method='mean',
                  variables=['incomeperperson', 'alcconsumption',
                             'armedforcesrate', 'breastcancerper100th',
                             'co2emissions', 'femaleemployrate', 'hivrate',
                             'internetuserate', 'lifeexpectancy',
                             'oilperperson', 'polityscore',
                             'relectricperperson', 'suicideper100th',
                             'employrate', 'urbanrate'])

In [10]:
df = imputer.transform(df)

In [11]:
df.isnull().sum()

country                 0
incomeperperson         0
alcconsumption          0
armedforcesrate         0
breastcancerper100th    0
co2emissions            0
femaleemployrate        0
hivrate                 0
internetuserate         0
lifeexpectancy          0
oilperperson            0
polityscore             0
relectricperperson      0
suicideper100th         0
employrate              0
urbanrate               0
dtype: int64

In [12]:
df

,country,incomeperperson,alcconsumption,armedforcesrate,breastcancerper100th,co2emissions,femaleemployrate,hivrate,internetuserate,lifeexpectancy,oilperperson,polityscore,relectricperperson,suicideper100th,employrate,urbanrate
0,Afghanistan,8740.965895,0.030000,0.570000,26.80000,7.594400e+07,25.600000,1.935442,3.65,48.670000,1.484603,0.000000,1173.179044,6.680000,55.700000,24.04
1,Albania,1915.000000,7.290000,1.020000,57.40000,2.237473e+08,42.100000,1.935442,44.99,76.920000,1.484603,9.000000,636.340000,7.700000,51.400000,46.72
2,Algeria,2231.990000,0.690000,2.310000,23.50000,2.932109e+09,31.700000,0.100000,12.50,73.130000,0.420000,2.000000,590.510000,4.850000,50.500000,65.22
3,Andorra,21943.340000,10.170000,1.443963,37.40289,5.033262e+09,47.549438,1.935442,81.00,69.753979,1.484603,3.689441,1173.179044,5.360000,58.635955,88.92
4,Angola,1381.000000,5.570000,1.460000,23.10000,2.483580e+08,69.400000,2.000000,10.00,51.090000,1.484603,-2.000000,173.000000,14.550000,75.700000,56.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Vietnam,722.810000,3.910000,1.090000,16.20000,1.425435e+09,67.600000,0.400000,27.85,75.180000,1.484603,-7.000000,302.730000,11.650000,71.000000,27.84
209,West Bank and Gaza,8740.965895,6.689412,5.940000,37.40289,1.424133e+07,11.300000,1.935442,36.42,72.830000,1.484603,3.689441,1173.179044,9.640419,32.000000,71.90
210,"Yemen, Rep.",610.360000,0.200000,2.320000,35.10000,2.348647e+08,20.300000,1.935442,12.35,65.490000,1.484603,-2.000000,130.060000,6.270000,39.000000,30.64
211,Zambia,432.230000,3.560000,0.340000,13.00000,1.320257e+08,53.500000,13.500000,10.12,49.030000,1.484603,7.000000,168.620000,12.020000,61.000000,35.42


### Treat Duplicate Values

In [13]:
df.duplicated(keep='first').sum()

0

### Treat Outliers

In [14]:
df.columns

Index(['country', 'incomeperperson', 'alcconsumption', 'armedforcesrate', 'breastcancerper100th', 'co2emissions', 'femaleemployrate', 'hivrate', 'internetuserate', 'lifeexpectancy', 'oilperperson', 'polityscore', 'relectricperperson', 'suicideper100th', 'employrate', 'urbanrate'], dtype='object')

In [15]:
df.describe()

,incomeperperson,alcconsumption,armedforcesrate,breastcancerper100th,co2emissions,femaleemployrate,hivrate,internetuserate,lifeexpectancy,oilperperson,polityscore,relectricperperson,suicideper100th,employrate,urbanrate
count,213.000000,213.000000,213.000000,213.000000,2.130000e+02,213.000000,213.000000,213.000000,213.000000,213.000000,213.000000,213.000000,213.000000,213.000000,213.000000
mean,8740.965895,6.689412,1.443963,37.402890,5.033262e+09,47.549438,1.935442,35.632760,69.753979,1.484603,3.689441,1173.179044,9.640419,58.635955,56.769360
std,13466.912542,4.589345,1.498691,20.444759,2.493649e+10,13.364005,3.632101,26.368642,9.191003,0.987114,5.486034,1341.777091,5.963944,9.611959,23.275759
min,103.780000,0.030000,0.000000,3.900000,1.320000e+05,11.300000,0.060000,0.210000,47.790000,0.030000,-10.000000,0.000000,0.200000,32.000000,10.400000
25%,952.830000,3.230000,0.570000,23.100000,3.795000e+07,40.300000,0.200000,11.500000,65.490000,1.484603,1.000000,431.630000,5.630000,53.500000,37.340000
50%,3665.350000,6.689412,1.210000,35.000000,2.348647e+08,47.549438,1.200000,35.632760,72.230000,1.484603,3.689441,1173.179044,9.510000,58.635955,56.769360
75%,8740.965895,9.500000,1.443963,44.300000,2.421918e+09,53.600000,1.935442,51.960000,76.070000,1.484603,8.000000,1173.179044,11.980000,63.700000,73.500000
max,105147.440000,23.010000,10.640000,101.100000,3.342209e+11,83.300000,25.900000,95.640000,83.390000,12.230000,10.000000,11154.760000,35.750000,83.200000,100.000000


In [16]:
#windsorizer = Winsorizer(distribution='skewed',tail='both',fold=1.5, variables=[])

In [17]:
#windsorizer.fit(df)

In [18]:
#df2 = windsorizer.transform(df)

In [19]:
#df2

In [20]:
#df2.describe()

In [21]:
#windsorizer.left_tail_caps_

In [22]:
#windsorizer.right_tail_caps_

STEP 2: Run frequency distributions for your chosen variables and select columns, and possibly rows. 

In [23]:
df.dtypes

country                  object
incomeperperson         float64
alcconsumption          float64
armedforcesrate         float64
breastcancerper100th    float64
co2emissions            float64
femaleemployrate        float64
hivrate                 float64
internetuserate         float64
lifeexpectancy          float64
oilperperson            float64
polityscore             float64
relectricperperson      float64
suicideper100th         float64
employrate              float64
urbanrate               float64
dtype: object

In [24]:
df.country.nunique()

213

In [25]:
df.country.describe()

count          213
unique         213
top       Maldives
freq             1
Name: country, dtype: object

In [26]:
df.groupby("country").mean()

,incomeperperson,alcconsumption,armedforcesrate,breastcancerper100th,co2emissions,femaleemployrate,hivrate,internetuserate,lifeexpectancy,oilperperson,polityscore,relectricperperson,suicideper100th,employrate,urbanrate
country,,,,,,,,,,,,,,,
Afghanistan,8740.965895,0.030000,0.570000,26.80000,7.594400e+07,25.600000,1.935442,3.65,48.670000,1.484603,0.000000,1173.179044,6.680000,55.700000,24.04
Albania,1915.000000,7.290000,1.020000,57.40000,2.237473e+08,42.100000,1.935442,44.99,76.920000,1.484603,9.000000,636.340000,7.700000,51.400000,46.72
Algeria,2231.990000,0.690000,2.310000,23.50000,2.932109e+09,31.700000,0.100000,12.50,73.130000,0.420000,2.000000,590.510000,4.850000,50.500000,65.22
Andorra,21943.340000,10.170000,1.443963,37.40289,5.033262e+09,47.549438,1.935442,81.00,69.753979,1.484603,3.689441,1173.179044,5.360000,58.635955,88.92
Angola,1381.000000,5.570000,1.460000,23.10000,2.483580e+08,69.400000,2.000000,10.00,51.090000,1.484603,-2.000000,173.000000,14.550000,75.700000,56.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,722.810000,3.910000,1.090000,16.20000,1.425435e+09,67.600000,0.400000,27.85,75.180000,1.484603,-7.000000,302.730000,11.650000,71.000000,27.84
West Bank and Gaza,8740.965895,6.689412,5.940000,37.40289,1.424133e+07,11.300000,1.935442,36.42,72.830000,1.484603,3.689441,1173.179044,9.640419,32.000000,71.90
"Yemen, Rep.",610.360000,0.200000,2.320000,35.10000,2.348647e+08,20.300000,1.935442,12.35,65.490000,1.484603,-2.000000,130.060000,6.270000,39.000000,30.64


In [27]:
df.groupby("country").count().unstack()

                 country           
incomeperperson  Afghanistan           1
                 Albania               1
                 Algeria               1
                 Andorra               1
                 Angola                1
                                      ..
urbanrate        Vietnam               1
                 West Bank and Gaza    1
                 Yemen, Rep.           1
                 Zambia                1
                 Zimbabwe              1
Length: 3195, dtype: int64

In [28]:
df.to_csv("gapminderfinal.csv",index=False)